<a href="https://colab.research.google.com/github/hyeonji0401/CUDA_practice/blob/main/CUDA_based_matrix_multiplication_program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7.1 행렬곱셈이란?
- 행렬 A 크기가 m*n, B의 크기가 k*n일때 행렬 C의 크기는 m*n이 됨
- 본 장에서 작성하고자 하는 행렬 곱셈 프로그램은 하나의 스레드 블록으로 처리할 수 없는 대규모 행렬의 연산을 목적으로 함(m,n,k의 각 크기가 256이상인 경우

In [ ]:
%%cuda
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <chrono>
#include <iostream>

#define DO_CPU
#define DATA_TYPE int

#define SIZE_M (512*2)
#define SIZE_N (512*4)
#define SIZE_K (512*2)

template<class T> void allocNinitMem(T** p, long long size, double* memUsage = NULL);
bool compareMatrix(DATA_TYPE* _A, DATA_TYPE* _B, int _size);

/******************************************************************
* Complete this kernels
******************************************************************/
__global__ void MatMul(DATA_TYPE* matA, DATA_TYPE* matB, DATA_TYPE* matC, int m, int n, int k)
{
  unsigned int row = blockIdx.x*blockDim.x+threadIdx.x;
  unsigned int col = blockIdx.y*blockDim.y+threadIdx.y;

	if(col>=n||row>=m) return;
	unsigned int index = row*n + col;
	matC[index]=0;
  for(int i=0; i<k; i++){
      matC[index]+=matA[row*k+i]*matB[i*n+col];
  }

}


int main(int argc, char* argv[])
{

	// set matrix size
	int m, n, k;
	m = SIZE_M;
	n = SIZE_N;
	k = SIZE_K;

	printf("Size : A = (%d by %d), B = (%d by %d), C = (%d by %d)\n", m, k, k, n, m, n);

	int sizeA = m * k;
	int sizeB = k * n;
	int sizeC = m * n;

	// Make matrix
	DATA_TYPE* A = NULL, * B = NULL;
	allocNinitMem<DATA_TYPE>(&A, sizeA);
	allocNinitMem<DATA_TYPE>(&B, sizeB);

	DATA_TYPE* Ccpu = NULL, * Cgpu = NULL;
	allocNinitMem<DATA_TYPE>(&Ccpu, sizeC);
	allocNinitMem<DATA_TYPE>(&Cgpu, sizeC);

	// generate input matrices
	for (int i = 0; i < sizeA; i++) A[i] = ((rand() % 10) + ((rand() % 100) / 100.0));
	for (int i = 0; i < sizeB; i++) B[i] = ((rand() % 10) + ((rand() % 100) / 100.0));

	// CPU algorithm
	auto hostStart = std::chrono::high_resolution_clock::now();
	for (int row = 0; row < m; row++) {
		for (int col = 0; col < n; col++) {
			int cIndex = row * n + col;
			Ccpu[cIndex] = 0;
			for (int i = 0; i < k; i++)
				Ccpu[cIndex] += (A[row * k + i] * B[i * n + col]);
		}
	}
	printf("CPU finished!\n");
	auto hostEnd = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> hostElapsed = hostEnd - hostStart;


	auto GPUStart = std::chrono::high_resolution_clock::now();
	/******************************************************************
	* Write your codes for GPU algorithm from here
	******************************************************************/
	DATA_TYPE* dA, * dB, * dC;

	// 1. Allocate device memory for dA, dB, dC
	// Hint: cudaMalloc, cudaMemset
  cudaMalloc(&dA, sizeof(DATA_TYPE)*sizeA); cudaMemset(dA, 0, sizeof(DATA_TYPE)*sizeA);
  cudaMalloc(&dB, sizeof(DATA_TYPE)*sizeB); cudaMemset(dB, 0, sizeof(DATA_TYPE)*sizeB);
  cudaMalloc(&dC, sizeof(DATA_TYPE)*sizeC); cudaMemset(dC, 0, sizeof(DATA_TYPE)*sizeC);
	auto h2dStart = std::chrono::high_resolution_clock::now();

	// 2. Send(Copy) the input matrices to GPU (A -> dB, B -> dB)
	// Hint: cudaMemcpy
  cudaMemcpy(dA, A, sizeof(DATA_TYPE)*sizeA, cudaMemcpyHostToDevice);
  cudaMemcpy(dB, B, sizeof(DATA_TYPE)*sizeB, cudaMemcpyHostToDevice);

	auto h2dEnd = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> h2dElapsed = h2dEnd - h2dStart;

	// 3. Set the thread layout
	//
	dim3 gridDim(ceil((float)SIZE_M/32), ceil((float)SIZE_N/32));
	dim3 blockDim(32, 32);

	printf("Grid(%d, %d), Block(%d, %d)\n", gridDim.x, gridDim.y, blockDim.x, blockDim.y);

	auto kernelStart = std::chrono::high_resolution_clock::now();

	// 4. Kernel call
	MatMul <<< gridDim, blockDim >>> (dA, dB, dC, m, n, k);

	cudaDeviceSynchronize(); // this is synchronization for mearusing the kernel processing time
	auto kernelEnd = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> kernelElapsed = kernelEnd - kernelStart;

	auto d2hStart = std::chrono::high_resolution_clock::now();

	//5. Get(copy) the result from GPU to host memory (dC -> Cgpu)
	// Hint: cudaMemcpy
  cudaMemcpy(Cgpu, dC, sizeC* sizeof(DATA_TYPE), cudaMemcpyDeviceToHost);

	auto d2hEnd = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> d2hElapsed = d2hEnd - d2hStart;

	// 6. Release device memory space (dA, dB, dC)
	// Hint: cudaFree
  cudaFree(dA);
  cudaFree(dB);
  cudaFree(dC);


	/******************************************************************
	******************************************************************/
	auto GPUEnd = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> GPUElapsed = GPUEnd - GPUStart;

  std::cout << "Host time: " << hostElapsed.count() << " ms" << std::endl;
  std::cout<<"Host -> Device: " << h2dElapsed.count() << " ms" << std::endl;
  std::cout<<"Kernel: " << kernelElapsed.count() << " ms" << std::endl;
  std::cout<<"Device -> Host: " << d2hElapsed.count() << " ms" << std::endl;
  std::cout << "GPU time: " << GPUElapsed.count() << " ms" << std::endl;

	compareMatrix(Ccpu, Cgpu, sizeC);

	delete A;
	delete B;
	delete Ccpu;
	delete Cgpu;

	return 0;
}


// Utility functions
bool compareMatrix(DATA_TYPE* _A, DATA_TYPE* _B, int _size)
{
	bool isMatched = true;
	for (int i = 0; i < _size; i++) {
		if (_A[i] != _B[i]) {
			printf("[%d] not matched! (%f, %f)\n", i, (double)_A[i], (double)_B[i]);
			getchar();
			isMatched = false;
		}
	}
	if (isMatched)
		printf("Results are matched!\n");
	else
		printf("Results are not matched!!!!!!!!!!!\n");

	return isMatched;
}

template<class T>
void allocNinitMem(T** p, long long size, double* memUsage) {
	*p = new T[size];
	memset(*p, 0, sizeof(T) * size);

	if (memUsage != NULL) {
		*memUsage += sizeof(T) * size;
	}
}

Size : A = (1024 by 1024), B = (1024 by 2048), C = (1024 by 2048)
CPU finished!
Grid(64, 32), Block(32, 32)
Host time: 26359.5 ms
Host -> Device: 3.0577 ms
Kernel: 100.398 ms
Device -> Host: 3.52164 ms
GPU time: 229.594 ms
Results are matched!



# 오답 노트



```
dim3 gridDim(ceil((float)SIZE_M/32), ceil((float)SIZE_N/32));
```
- 인자는 순서대로 X,Y,Z를 나타냄
- M은 결과 행렬의 행이고 N은 결과 행렬의 열이므로 커널 코드에서 row에는 X차원과 매칭하고 col에는 Y차원과 매칭하면 됨
- 인자 순서가 바뀔 경우 커널 코드에서도 반대로 매칭해야함
- SIZE_M 앞의 캐스트 연산자는 올림을 하기 위한 것으로 꼭 필요함



```
matC[index]=0;
```
- 행렬의 합을 담을 것이므로 혹시 모를 쓰레기 값을 대비하여 초기화해주는 것이 중요함



```
cudaMemcpy(Cgpu, dC, sizeC* sizeof(DATA_TYPE), cudaMemcpyDeviceToHost);
```
- 값을 옮길 때 sizeof(DATA_TYPE)을 해주지 않아 결과가 항상 맞지 않았음
- 빼먹은 걸 뒤늦게 찾아서 시간을 많이 소모함
- 데이터 크기를 올바르게 설정해야 올바른 크키대로 복사되어 올바른 비교를 할 수 있으므로 주의 할 것





# 7.2 스레드 레이아웃 설정
1.데이터를 읽는 행렬 A,B 기준
- 읽기 연산의 반복

1) 행렬 A의 각 행 또는 행렬 B에서 각 열을 하나의 스레드가 처리하게 하는 것
- 병렬 처리에 참여할 수 있는 스레드의 수가 행렬 A의 행 개수 또는 행렬 B의 열 개수로 제한됨

  => 병렬 처리 정도(degree of parallelism)가 낮아짐
- GPU 병렬 처리 능력을 최대한 활용하려면 많은 스레드를 사용하는 것이 중요하므로 입력 행렬 중 하나만을 기준으로 삼는 레이아웃

2) 두 입력 행렬을 동시에 고려
- 모든 A와 B 쌍의 곱을 동시에 병렬로 처리
- m * n * k 스레드를 사용하게 됨
- 행렬의 C입장에서 보면 여러 개의 스레드가 동시에 접근해서 갱신해야하는 상황이 발생함
 - 동기화 문제 발생 ( 여러 스레드가 쓰기 연산 시 이상한 값 생성 가능성)
 - 해결을 위해 동기화 기법 사용시 직렬화되어 성능이 떨어짐
2.결과가 저장되는 행렬 C 기준
- 쓰기 연산의 반복
- 쓰기 연산에는 동기화 문제가 발생하므로 쓰기 영역을 분할하여 스레드들에게 분배하는 방법
 - 행렬 C의 각 원소를 스레드들에게 분배
 - 동시 접근은 읽기 연산에서만 발생

 => 행렬은 2차원 데이터로 레이아웃 또한 2차원으로 잡는 것이 일반적임



